<a href="https://colab.research.google.com/github/Jazielinho/aprendizaje_refuerzo/blob/main/cap_2_formalizacion_aprendizaje_refuerzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1 Proceso de decisión de Markov

## Recapitulación de conceptos

Un agente influye en el comportamiento observado de un entorno eligiendo acciones. El objetivo es elegir las acciones que maximicen la recompensa.

El agente y el entorno interactúan en una secuencia de *time step* discretos.

En el time step $t$:
* $s$: estado del entorno ($s_{t})$
* $a$: acción del agente ($a_{t})$

La acción $a$ recibe en el siguiente time step $t + 1$:
* $r'$ recompensa ($r_{t+1})$
* $s'$: el entorno cambia al nuevo estado ($s_{t+1})$

El conjunto del estado, acción, recompensa y nuevo estado se llama *experiencia*

![](https://miro.medium.com/max/1400/1*x-Bj67wALCbWrtSNhb5h0A.png)

## Proceso de decisión de Markov
O *Markov Decision Process* (MDP), marco matemático para resolver algunos de los problemas de aprendizaje por refuerzo (RL)

Consiste en una tupla de 5 elementos: $<S, A, R, P, γ>$

* $S$: conjunto de estados.
* $A$: conjunto de acciones.
* $R$: función de recompensa.
* $P$: función de transición.
* $\gamma$: factor de descuento.

# 2.2 Piezas de un proceso de decisión de Markov

In [1]:
import gym

entorno  = gym.make('FrozenLake-v0', is_slippery=False)

## Estados

U *observación* es una configuración única y autónoma del problema.

*espacio de estado*: conjunto de todos los estados posibles, puede ser infinito.

In [2]:
''' Espacio de estados '''
print(f"Espacio de estados: {entorno.observation_space}")

Espacio de estados: Discrete(16)


Estados particulares, ejemplo, estado de partida o estados terminales. En Frozen Lake: *estado inicial* (0), 5 *estados terminales* (5, 7, 11, 12, 15)

![](https://miro.medium.com/max/1400/1*yTHeP5HSKXCpnv9wbgGIoQ.png)

*Propiedad de Markov*: el futuro depende solo del presente y no del pasado. El estado actual que obtenemos del entorno contiene lo que se necesita para decidir el estado futuro cuando se realiza una acción.

La mayoría de los métodos de RL están diseñados para aprovechar esta suposición.

Ejemplo, Frozen Lake, estado actual: 2, el agente solo puede realizar la transición a 1, 2, 3, o 6, independientemente del estado anterior.

## Acciones
Un conjunto de acciones $A$ dependen del estado, por lo que pueden variar en cada *time step*.
El agente elige una acción, esto influye en el entorno y el entorno cambia el estado.

In [3]:
''' 4 acciones '''
''' izquierda, abajo, derecha, arriba '''
print(f"Espacio de acciones: {entorno.action_space}")

Espacio de acciones: Discrete(4)


* Espacio de acción discreto: el espacio de acción consta de acciones discretas (ejemplo, Frozen Lake)
* Espacio de acción continua: el espacio de acción compone de acciones continuas (ejemplo, conducción)

## Función de transición

$P$, representa la probabilidad de moverse de un estado a otro. Forma parte del MDP, no es conocida por el agente, representa la dinámica de un *time step* del entorno.

$P(s'|a)$ Quiere expresar que el siguiente estado $s'$ depende solo del estado anterior $s$ y la acción $a$.

## Recompensa

$R(s, a)$ o $R(s, a, s')$ representa la recompensa que nuestro agente obtiene durante la transición de estado $s$ al estado $s'$ al realizar una acción $a$. Es un número real. Este valor dice al agente cuán bien se ha comportado.

En Frozen Lake, la función de recompensa es "+1" cuando llega al estado final y "0" en cualquier otro caso.

*return*: indica la recompensa acumulada.

## Factor de descuento
* *tareas epísodicas*: cuando las tareas tienen un final natural. La secuencia *time step* se llama *episodio*.
* *tareas continuas*: cuando no tienen un final natural.

En el caso de tareas continuas necesitamos una forma de descontar el valor de las recompensas a lo largo del tiempo.

*factor de descuento* $γ \in [0, 1]$, ajusta la importancia de las recompensas a lo largo del tiempo.

Cuanto más tarde reciba las recompensas el agente, menos atractivo será para él obtenerla.

# 2.3 Entornos deterministas y estocásticos

El agente conocerá el estado, acciones, recompensas (más factor de descuento), pero la función de transición por lo general será desconocida.

Los estados y recompensas pueden ser variables aleatorias porque la mayoría de los entornos son estocásticos.

Dos tipos: *entornos deterministas* y *entornos estocásticos*

## Entorno determinista

In [4]:
import gym

#is_slippery=False indica no resbaladizo. Entorno determinista
entorno  = gym.make('FrozenLake-v0', is_slippery=False)

La probabilidad en el *time step* $t$ del siguiente estado $s_{t+1}$, dado el estado actual $s_t$ y la acción $a_t$ es siempre 1.

$p(s_{t+1}|s_t, a_t) = 1$

In [5]:
entorno.env.P

{0: {0: [(1.0, 0, 0.0, False)],
  1: [(1.0, 4, 0.0, False)],
  2: [(1.0, 1, 0.0, False)],
  3: [(1.0, 0, 0.0, False)]},
 1: {0: [(1.0, 0, 0.0, False)],
  1: [(1.0, 5, 0.0, True)],
  2: [(1.0, 2, 0.0, False)],
  3: [(1.0, 1, 0.0, False)]},
 2: {0: [(1.0, 1, 0.0, False)],
  1: [(1.0, 6, 0.0, False)],
  2: [(1.0, 3, 0.0, False)],
  3: [(1.0, 2, 0.0, False)]},
 3: {0: [(1.0, 2, 0.0, False)],
  1: [(1.0, 7, 0.0, True)],
  2: [(1.0, 3, 0.0, False)],
  3: [(1.0, 3, 0.0, False)]},
 4: {0: [(1.0, 4, 0.0, False)],
  1: [(1.0, 8, 0.0, False)],
  2: [(1.0, 5, 0.0, True)],
  3: [(1.0, 0, 0.0, False)]},
 5: {0: [(1.0, 5, 0, True)],
  1: [(1.0, 5, 0, True)],
  2: [(1.0, 5, 0, True)],
  3: [(1.0, 5, 0, True)]},
 6: {0: [(1.0, 5, 0.0, True)],
  1: [(1.0, 10, 0.0, False)],
  2: [(1.0, 7, 0.0, True)],
  3: [(1.0, 2, 0.0, False)]},
 7: {0: [(1.0, 7, 0, True)],
  1: [(1.0, 7, 0, True)],
  2: [(1.0, 7, 0, True)],
  3: [(1.0, 7, 0, True)]},
 8: {0: [(1.0, 8, 0.0, False)],
  1: [(1.0, 12, 0.0, True)],
  2: [(

Ejemplo de salida del diccionario:

0: {0: [(1.0, 0, 0.0, False)],}

Cada estado contiene un diccionario que mapea todas las acciones posibles.

* Primer 0: ubicación en la matriz.
* Segundo 0: dirección (izquierda, abajo, derecha, arriba)
* Lista:
  * 1.0: Probabilidad de ir al siguiente estado (igual a 1, entorno determinista)
  * 0: El siguiente estado (nueva ubicación en la matriz)
  * 0.0: recompensa al pasar al siguiente estado.
  * False: si el episocio termina allí o no.

![](https://miro.medium.com/max/1400/1*M7viRSF1GTwDV3twKWzGSw.png)

Programando al agente para realizar un buen plan (llegar a la celda final con éxito):

In [6]:
class Agente:
  def __init__(self):
    self.acciones = {'Izquierda': 0, 'Abajo': 1, 'Derecha': 2, 'Arriba': 3}
    self.buen_plan = 2 * ['Abajo'] + ['Derecha'] + ['Abajo'] + 2 * ['Derecha']
    self.step = 0
  
  def selecciona_accion(self):
    accion = self.buen_plan[self.step]
    self.step = (self.step + 1) % 6
    return self.acciones[accion]
  
  def reiniciar(self):
    ''' inicializamos el agente cada vez que es llamado '''
    self.step = 0

In [10]:
entorno = gym.make('FrozenLake-v0', is_slippery=False)

entorno.reset()
entorno.render()
esta_terminado = False
t = 0

agente = Agente()

while not esta_terminado:
  accion = agente.selecciona_accion()
  estado, recompensa, esta_terminado, _ = entorno.step(accion)
  entorno.render()
  t += 1

print(f"Último estado: {estado}")
print(f"Recompensa: {recompensa}")
print(f"time steps: {t}")


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Último estado: 15
Recompensa: 1.0
time steps: 6


In [15]:
''' Funcion que calcula el éxito del agente '''

def test(agente, entorno_test):
  entorno_test.reset()
  agente.reiniciar()

  esta_terminado = False
  t = 0

  while not esta_terminado:
    accion = agente.selecciona_accion()
    estado, recompensa, esta_terminado, _ = entorno_test.step(accion)
    t += 1
  
  return estado, recompensa, esta_terminado

In [16]:
''' Probando la función 1000 veces para calcular la proporción de aciertos '''

agente = Agente()
entorno = gym.make('FrozenLake-v0', is_slippery=False)
resuelto = 0

for episodio in range(1000):
  estado, recompensa, esta_terminado = test(agente, entorno)
  if estado == 15:
    resuelto += 1

print(f"Resuelto: {resuelto}, {resuelto/10}% de las veces")


Resuelto: 1000, 100.0%


## Entorno estocástico

La función de transición es un poco más compleja, ya que debe indicar las diversas probabilidades de acabar en diferentes estados que tiene cada acción.

Para Frozen Lake la función de transición se representa en una matriz 3D, donde cada elemento indica la probabilidad de transición de un estado de origen $s$ a un destino $s'$ dada una acción $a$

In [18]:
''' La función de transición en 3D '''
entorno = gym.make('FrozenLake-v0')
entorno.env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

La probabilidad de ir a la celda prevista es del 33%, 66% de ir a otras celdas. Si estamos al lado de una valla transparente y nos dirigimos hacia ella, no necesariamente volveremos a la misma celda.

In [19]:
''' Probando el Agente creado en este entorno estocástico '''

agente = Agente()
entorno = gym.make('FrozenLake-v0')
resuelto = 0

for episodio in range(1000):
  estado, recompensa, esta_terminado = test(agente, entorno)
  if estado == 15:
    resuelto += 1

print(f"Resuelto: {resuelto}, {resuelto/10}% de las veces")


Resuelto: 49, 4.9%


Si estamos en el estado 14, esta es a función de transición:

In [20]:
entorno = gym.make('FrozenLake-v0')
entorno.env.P[14]

{0: [(0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False)],
 1: [(0.3333333333333333, 13, 0.0, False),
  (0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True)],
 2: [(0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False)],
 3: [(0.3333333333333333, 15, 1.0, True),
  (0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 13, 0.0, False)]}

![](https://miro.medium.com/max/1400/1*yUMc-oRhXJoUBeNbKD3fcA.png)

Por ejemplo, estando en la celda 14: si elegimos la opción 1 (ir abajo), nos topamos con una valla transparente. Pero en lugar de volver a la misma celda 14, tenemos un tercio de probabilidad de lograrlo, con esa misma probabilidad (un tercio) podemos aparecer en la celda 13 o la celda 15.

# 2.4 Configuración del problema a resolver

## Episodio y trayectoria

$r=(s_0, a_0, r_1, s_1, a_1, r_2, s_2, ..., a_H, r_{H+1}, s_{H+1})$

## Retorno con descuento

$G_t = r_{t+1} + r_{t+2} + r_{t+3} + r_{t+4} + ... = \sum_{k=t+1}^{T}r_k$

$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \gamma^3 r_{t+4} + ... = \sum_{k=t+1}^{T}\gamma^{k-t-1} r_k$

## Política